# eStore

In [1]:
from requests_oauthlib import OAuth1
from requests_oauthlib import OAuth1Session
import requests
import hmac

In [2]:
import pandas as pd
import numpy as np

In [3]:
client_key = ''
client_secret = ''
resource_owner_key=''
resource_owner_secret=''

url=''

headeroauth = OAuth1(
    realm = url,
    client_key = client_key,
    client_secret = client_secret,
    resource_owner_key = resource_owner_key,
    resource_owner_secret = resource_owner_secret,
)

headers = {
    'User-Agent': '',
    'Content-Type': 'application/json',
}

response = requests.get(
    url,
    auth = headeroauth,
    headers=headers
    )

In [4]:
client_key_dev = ''
client_secret_dev = ''
resource_owner_key_dev=''
resource_owner_secret_dev=''

In [5]:
deviceList=response.json()

In [6]:
import json
with open('result.json', 'w') as fp:
    json.dump(deviceList[0], fp)

In [7]:
deviceList[0]['items'][0].keys()

dict_keys(['id', 'name', 'is_prepaid_bundle', 'plan_tab', 'categories', 'device_combo_data', 'sku', 'operating_system', 'brand', 'mostpopular', 'price', 'isIOTDevice', 'bundle_price', 'base_plan', 'base_plan_title', 'default_plan', 'default_plan_sku', 'plan_min_price', 'device_min_price', 'discounted_device_min_price', 'image', 'full_width_banner', 'pre_order_data', 'discount_promotion_data', 'is_lifestyle', 'lifestyle_hat_text1', 'lifestyle_hat_text2', 'is_easy_phone', 'is_device_bundle', 'is_rent', 'price_start_from', 'is_premium_plan', 'visibility', 'promotion_badge', 'part_number', 'promotiondetails', 'specifications', 'launch_date', 'is_comparable', 'stock_highlighter'])

In [8]:
selCols=['id', 'name', 'sku', 'base_plan']

In [9]:
deviceSkus=list(set([device['sku'] for device in deviceList[0]['items']]))

In [10]:
len(deviceSkus)

140

In [11]:
def dictListFilter(ld,k,v):
    newDict={}
    for d in ld:
        if k in d:
            if d[k]==v:
                newDict=d
                break
    return(newDict)

In [ ]:
devices = []
url_det='/'

plan_mapping = {
 '': {'plan_part_number':'','pass_part_number':None},
 '': {'plan_part_number':'','pass_part_number':''},
 '': {'plan_part_number':'','pass_part_number':''},
 '': {'plan_part_number':'','pass_part_number':''},
 '': {'plan_part_number':'','pass_part_number':''},
 '': {'plan_part_number':'','pass_part_number':''},
 '': {'plan_part_number':'','pass_part_number':''},
 '': {'plan_part_number':'','pass_part_number':''},
 '': {'plan_part_number':'','pass_part_number':''},
 '': {'plan_part_number':'','pass_part_number':''}
    }

for dev in deviceSkus:
    print(dev)
    url_dev=url_det+dev
    headeroauth_dev = OAuth1(
        realm = url_dev,
        client_key = client_key_dev,
        client_secret = client_secret_dev,
        resource_owner_key = resource_owner_key_dev,
        resource_owner_secret = resource_owner_secret_dev,
    )

    headers_dev = {
        'User-Agent': '',
        'Content-Type': 'application/json',
    }

    response_dev = requests.get(
        url_dev,
        auth = headeroauth_dev,
        headers=headers_dev
        )
    
    data_dev=response_dev.json()
    
    if (type(data_dev) is not dict):
        if ('items' in data_dev[0]):
            a={ your_key: data_dev[0]['items']['basic_details'][your_key] 
                               for your_key in ['name','sku','order_brand','start_date']}
            b=[{'memory':x1,
                'part_number':x2,
                'rrp_rm_strick_price':x3,
                'pass_sku':x4,
                'upfront_price':x5,
                'upfront_duration':x6,
                'device_price':x7,
                'penalty_price':x8} 
               for x1,x2,x3,x4,x5,x6,x7,x8 in 
                   list({str(i):i.values() 
                     for i in [{'memory':i['memory'],
                                'part_number':i['part_number'],
                                'rrp':i['rrp'],
                                'sku':j['sku'],
                                'upfront_price':j['prices'][0]['upfront_price'],
                                'upfront_duration':i['upfront_installment'],
                                'device_price':j['prices'][0]['device_price'],
                                'penalty_price':j['prices'][0]['penalty_price']} 
                        for i in data_dev[0]['items']['associated_product'] if i['saleable_plans'] 
                            for j in i['saleable_plans']]}.values())]
            
            for i in b:
                devices.append({**a,**i})

for i in devices:
    if i['pass_sku'] in plan_mapping.keys():
        i['plan_part_number']=plan_mapping[i['pass_sku']]['plan_part_number']
        i['pass_part_number']=plan_mapping[i['pass_sku']]['pass_part_number']

In [13]:
df=pd.DataFrame(devices)

In [14]:
df['plan_title'] = df.pass_sku.str.split('-').str[0]+' '+ df.pass_sku.str.split('-').str[1]
df['plan_part_number'] = df.plan_part_number
df['plan_sku'] = df.pass_sku
df['pass_name'] = df.pass_sku.str.split('-').str[2]
df['pass_sku'] = df.pass_sku
df['pass_part_number'] = df.pass_part_number

In [15]:
df=df[['name','sku','order_brand','rrp_rm_strick_price','start_date','part_number','plan_title','plan_part_number','plan_sku','pass_name','pass_sku','pass_part_number','upfront_price','device_price',	'penalty_price']]

In [16]:
df.loc[df.pass_sku=='','plan_title']=''
df.loc[df.pass_sku=='','pass_name']=None

In [ ]:
df[df.plan_title==''].head()

In [18]:
df.to_csv('from_eStore.csv',index= False)